In [1]:
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from functools import reduce 
from pyspark.sql import DataFrame
import pyspark.pandas as ps

spark = SparkSession.builder.getOrCreate()

In [2]:
import pandas as pd
import numpy as np
import plotly.io as pio
import plotly.express as px
import plotly.figure_factory as ff
import matplotlib.pyplot as plt
import matplotlib.patches
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from math import sqrt
from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen
from os import listdir
import warnings

warnings.filterwarnings('ignore')

In [3]:
with open('folder.txt', 'r') as folder:
    lines = folder.readlines()
    
folder_archivos =  ' '.join(lines)

csv_files = []

In [4]:
def find_csv_filenames( path_to_dir, suffix=".csv" ):
    filenames = listdir(path_to_dir)
    return [ filename for filename in filenames if filename.endswith( suffix ) ]

csv_files = [l for l in find_csv_filenames(folder_archivos, suffix=".csv" ) if 'autos' in l]

In [5]:
with open('urls.txt', 'r') as folder_urls:
    lines_urls = folder_urls.readlines()

if len(csv_files) == 0:
    zipurls = lines_urls

    for zipurl in zipurls:
        with urlopen(zipurl) as zipresp:
            with ZipFile(BytesIO(zipresp.read())) as zfile:
                zfile.extractall(folder_archivos)

In [6]:
list_data_frames = []

for file in csv_files:
    df_new = spark.read.csv(file, header = True, sep = ',')
    list_data_frames.append(df_new)

In [23]:
def unionAll(*dfs):
    return reduce(DataFrame.unionAll, dfs)

In [24]:
df = unionAll(*list_data_frames)

In [26]:
df = df.groupBy(['tramite_fecha','registro_seccional_provincia']).agg(count("registro_seccional_provincia").alias("count"))

In [29]:
df.toPandas().to_csv('df.csv')

In [55]:
#df = ps.DataFrame(df)

In [57]:
#df = df.pivot(index='tramite_fecha', columns='registro_seccional_provincia', values='count').sort_index()

In [58]:
#df.columns.name = None              
#df = df.reset_index() 
#df = df.rename_axis(None, axis=1)